In [1]:
import cv2 as cv

In [2]:
import numpy as np

In [3]:
import os

In [4]:
import csv

In [6]:
import tensorflow as tf

In [7]:
from keras import backend as K

Using TensorFlow backend.


In [8]:
from sklearn.model_selection import train_test_split

In [9]:
from keras.layers import Input, Lambda, Dense, Dropout, ZeroPadding2D, Conv2D, MaxPooling2D, Flatten, BatchNormalization

In [10]:
from keras.utils import to_categorical

In [11]:
from keras.models import Model

In [12]:
from keras.optimizers import SGD

In [13]:
from keras.callbacks import TensorBoard, EarlyStopping

In [14]:
train_path = ("./image_train_10classes")

In [15]:
train_list = []

In [16]:
path_list = os.listdir(train_path)

In [17]:
path_list.sort(key = lambda x:int(x[5:-6]))

In [19]:
for filename in path_list:
    filename = train_path + '/' + filename
    img = cv.imread(filename,cv.IMREAD_COLOR)
    img = np.resize(img,(227,227,3))
    img = img.astype('float32')
    train_list.append(img)

In [20]:
train_array = np.array(train_list)

In [21]:
print(train_array)

[[[[203. 123.  58.]
   [204. 126.  60.]
   [203. 127.  61.]
   ...
   [230. 156.  92.]
   [229. 155.  91.]
   [230. 156.  92.]]

  [[232. 158.  94.]
   [231. 157.  93.]
   [230. 156.  92.]
   ...
   [255. 190. 122.]
   [255. 191. 123.]
   [255. 192. 121.]]

  [[254. 194. 118.]
   [254. 195. 116.]
   [254. 192. 121.]
   ...
   [222. 151.  84.]
   [221. 150.  83.]
   [221. 150.  83.]]

  ...

  [[255. 188. 119.]
   [255. 188. 119.]
   [255. 186. 119.]
   ...
   [229. 154.  80.]
   [229. 154.  78.]
   [229. 151.  84.]]

  [[229. 153.  81.]
   [230. 155.  81.]
   [230. 152.  85.]
   ...
   [249. 178. 114.]
   [250. 180. 113.]
   [250. 181. 112.]]

  [[250. 180. 113.]
   [249. 179. 112.]
   [250. 180. 113.]
   ...
   [124.  81.   8.]
   [148.  96.   2.]
   [170. 111.   1.]]]


 [[[ 45.  37.  38.]
   [ 31.  23.  24.]
   [ 28.  20.  21.]
   ...
   [ 67.  56.  66.]
   [ 71.  60.  70.]
   [ 75.  64.  74.]]

  [[ 75.  64.  74.]
   [ 70.  61.  71.]
   [ 69.  60.  70.]
   ...
   [ 33.  22.  25.]
 

In [22]:
print(train_array.shape)

(13000, 227, 227, 3)


In [23]:
import scipy.io as sio
load_image_Average = sio.loadmat('image_Average.mat')
image_Average = load_image_Average['img_AverageImage']
print(image_Average)

[[[117.42131  117.691475 110.2155  ]
  [117.78874  118.09067  110.54358 ]
  [118.06558  118.34228  110.774086]
  ...
  [118.2967   118.58421  111.0478  ]
  [118.054085 118.31674  110.85942 ]
  [117.70546  117.95547  110.54898 ]]

 [[117.74736  118.01939  110.49082 ]
  [118.40584  118.73573  111.102875]
  [118.74161  119.05518  111.39909 ]
  ...
  [118.96284  119.28572  111.673325]
  [118.646385 118.94318  111.40893 ]
  [117.958755 118.23849  110.75787 ]]

 [[117.92895  118.18693  110.607086]
  [118.68223  118.99819  111.300674]
  [119.09198  119.397385 111.657486]
  ...
  [119.28709  119.59832  111.91848 ]
  [118.8726   119.15427  111.56607 ]
  [118.10415  118.36356  110.83473 ]]

 ...

 [[114.09992  110.46717   98.72192 ]
  [114.75238  111.15133   99.25297 ]
  [115.10269  111.45416   99.49157 ]
  ...
  [114.95822  111.295     99.53013 ]
  [114.622185 110.95591   99.31079 ]
  [113.96866  110.33514   98.77194 ]]

 [[113.938194 110.30937   98.59753 ]
  [114.46761  110.86142   99.02909 ]


In [24]:
for i in range(len(train_array)):
    train_array[i] = train_array[i] - image_Average

In [25]:
print(train_array)

[[[[ 8.55786896e+01  5.30852509e+00 -5.22154999e+01]
   [ 8.62112579e+01  7.90933228e+00 -5.05435791e+01]
   [ 8.49344177e+01  8.65772247e+00 -4.97740860e+01]
   ...
   [ 1.11703300e+02  3.74157867e+01 -1.90477982e+01]
   [ 1.10945915e+02  3.66832581e+01 -1.98594208e+01]
   [ 1.12294540e+02  3.80445328e+01 -1.85489807e+01]]

  [[ 1.14252640e+02  3.99806137e+01 -1.64908218e+01]
   [ 1.12594162e+02  3.82642670e+01 -1.81028748e+01]
   [ 1.11258392e+02  3.69448166e+01 -1.93990936e+01]
   ...
   [ 1.36037170e+02  7.07142792e+01  1.03266754e+01]
   [ 1.36353607e+02  7.20568237e+01  1.15910721e+01]
   [ 1.37041245e+02  7.37615128e+01  1.02421265e+01]]

  [[ 1.36071045e+02  7.58130722e+01  7.39291382e+00]
   [ 1.35317780e+02  7.60018082e+01  4.69932556e+00]
   [ 1.34908020e+02  7.26026154e+01  9.34251404e+00]
   ...
   [ 1.02712914e+02  3.14016800e+01 -2.79184799e+01]
   [ 1.02127403e+02  3.08457336e+01 -2.85660706e+01]
   [ 1.02895851e+02  3.16364365e+01 -2.78347321e+01]]

  ...

  [[ 1.40900

In [26]:
print(train_array.shape)

(13000, 227, 227, 3)


In [27]:
label_path = ("./label.csv")

In [28]:
label_list = []

In [29]:
reader = csv.reader(open(label_path))

In [30]:
for id, label in reader:
    label_list.append(label)

In [32]:
label_array = np.array(label_list)

In [33]:
print(label_array)

['balloon' 'balloon' 'balloon' ... 'banana' 'banana' 'banana']


In [34]:
label_array = np.char.replace(label_array,'balloon'   ,'0')
label_array = np.char.replace(label_array,'candle'    ,'1')
label_array = np.char.replace(label_array,'desk'      ,'2')
label_array = np.char.replace(label_array,'horse cart','3')
label_array = np.char.replace(label_array,'notebook'  ,'4')
label_array = np.char.replace(label_array,'sunglass'  ,'5')
label_array = np.char.replace(label_array,'tank'      ,'6')
label_array = np.char.replace(label_array,'teddy'     ,'7')
label_array = np.char.replace(label_array,'umbrella'  ,'8')
label_array = np.char.replace(label_array,'banana'    ,'9')

In [35]:
label_array = label_array.astype('int')

In [36]:
print(label_array)

[0 0 0 ... 9 9 9]


In [37]:
label = to_categorical(label_array, 10)

In [38]:
print(label)

[[1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]]


In [39]:
x_train, x_test = train_test_split(train_array, test_size = 0.2, random_state = 6)

In [40]:
y_train, y_test = train_test_split(label, test_size = 0.2, random_state = 6)

In [41]:
def Quantify16_num(num):
    
    if num < 0 :
        is_neg_num = 1
        num =  -num
    else :
        is_neg_num = 0

        
    num = float(num)
    num = str(num)
    if num.find('e') != -1 :
        num = '{:.6f}'.format(float(num))
    num_int, num_dec = num.split(".")
    digitAfterPoint = len(num_dec)
    num_int = int(num_int)
    num_dec = float(num_dec)
    num_dec = num_dec/pow(10,digitAfterPoint)
    
    
    if num_int != 0 :
        num_int = bin(num_int).lstrip("0b")
        num_int = float(num_int)
        
        
    num_whole = num_int

    if num_dec >= 0.5 :
        num_whole = num_whole + 0.1
        num_dec = num_dec - 0.5

    if num_dec >= 0.25 :
        num_whole = num_whole + 0.01
        num_whole = round(num_whole,2)
        num_dec = num_dec - 0.25

    if num_dec >= 0.125 :
        num_whole = num_whole + 0.001
        num_whole = round(num_whole,3)
        num_dec = num_dec - 0.125

    if num_dec >= 0.0625 :
        num_whole = num_whole + 0.0001
        num_whole = round(num_whole,4)
        num_dec = num_dec - 0.0625

    if num_dec >= 0.03125 :
        num_whole = num_whole + 0.00001
        num_whole = round(num_whole,5)
        num_dec = num_dec - 0.03125

    if num_dec >= 0.015625 :
        num_whole = num_whole + 0.000001
        num_whole = round(num_whole,6)

        
    num_whole = str(num_whole)
    num_whole = float(num_whole)
    num_whole = str(num_whole)
    if num_whole.find('e') != -1 :
        num_whole = '{:.6f}'.format(float(num_whole))
    num_int, num_dec = num_whole.split(".")
    if num_dec.find('e') != -1 :
        num_dec = ('{:.6f}'.format(float(num_dec)))[2:7]
    num_dec = "{:0<6}".format(num_dec)
    num_int = int(num_int,2)
    num_dec0 = float(num_dec[0])
    num_dec1 = float(num_dec[1])
    num_dec2 = float(num_dec[2])
    num_dec3 = float(num_dec[3])
    num_dec4 = float(num_dec[4])
    num_dec5 = float(num_dec[5])
    num_dec = num_dec0*0.5 + num_dec1*0.25 + num_dec2*0.125 + num_dec3*0.0625 + num_dec4*0.03125 + num_dec5*0.015625
    num_whole = num_int + num_dec

    
    if is_neg_num == 1 :
        num_whole = -num_whole
        
    
    return num_whole

In [42]:
y = Quantify16_num(-1.96221848e+01)
print(y)

-19.609375


In [54]:
def Quantify16(input_tensor):
    
    input_array = K.eval(input_tensor)
    
    temp4 = []
    for i in range(len(input_array)):
        temp3 = []
        for j in range(len(input_array[i])):
            temp2 = []
            for k in range(len(input_array[i][j])):
                temp1 = []
                for z in range(len(input_array[i][j][k])):
                    a = Quantify16_num(input_array[i][j][k][z])
                    temp1.append(a)
                temp2.append(temp1)
            temp3.append(temp2)
        temp4.append(temp3)
        
        temp4 = np.array(temp4)
        return temp4

In [44]:
def Quantify10_num(num):
    
    if num < 0 :
        is_neg_num = 1
        num =  -num
    else :
        is_neg_num = 0
        
    num_int, num_dec = str(num).split(".")
    num_int = int(num_int)
    if num_int != 0 :
        num_int = bin(num_int).lstrip("0b")
    num_int = str(num_int)
    num_int_len = len(num_int)
    
    if num_int_len <= 9 :
        num_int = num_int
    else :
        num_int = num_int[0:9]

    num_int = int(num_int,2)
    
    if is_neg_num == 1 :
        num_int = -num_int

    return num_int

In [45]:
def Quantify10(input_tensor):
    
    temp4 = []
    for i in range(len(input_tensor)):
        temp3 = []
        for j in range(len(input_tensor[i])):
            temp2 = []
            for k in range(len(input_tensor[i][j])):
                temp1 = []
                for z in range(len(input_tensor[i][j][k])):
                    a = Quantify10_num(input_tensor[i][j][k][z])
                    temp1.append(a)
                temp2.append(temp1)
            temp3.append(temp2)
        temp4.append(temp3)
        
        temp4 = np.array(temp4)
        return temp4

In [46]:
def Quantify9_num(num):
    
    if num < 0 :
        is_neg_num = 1
        num =  -num
    else :
        is_neg_num = 0
        
    num_int, num_dec = str(num).split(".")
    num_int = int(num_int)
    if num_int != 0 :
        num_int = bin(num_int).lstrip("0b")
    num_int = str(num_int)
    num_int_len = len(num_int)
    
    if num_int_len <= 8 :
        num_int = num_int
    else :
        num_int = num_int[0:8]

    num_int = int(num_int,2)
    
    if is_neg_num == 1 :
        num_int = -num_int
    
    return num_int   

In [47]:
def Quantify9(input_tensor):
    
    temp4 = []
    for i in range(len(input_tensor)):
        temp3 = []
        for j in range(len(input_tensor[i])):
            temp2 = []
            for k in range(len(input_tensor[i][j])):
                temp1 = []
                for z in range(len(input_tensor[i][j][k])):
                    a = Quantify9_num(input_tensor[i][j][k][z])
                    temp1.append(a)
                temp2.append(temp1)
            temp3.append(temp2)
        temp4.append(temp3)
        
        temp4 = np.array(temp4)
        return temp4

In [48]:
def Quantify8_num(num):
    
    if num < 0 :
        is_neg_num = 1
        num =  -num
    else :
        is_neg_num = 0
        
    num_int, num_dec = str(num).split(".")
    num_int = int(num_int)
    if num_int != 0 :
        num_int = bin(num_int).lstrip("0b")
    num_int = str(num_int)
    num_int_len = len(num_int)
    
    if num_int_len <= 7 :
        num_int = num_int
    else :
        num_int = num_int[0:7]

    num_int = int(num_int,2)
    
    if is_neg_num == 1 :
        num_int = -num_int
    
    return num_int 

In [49]:
def Quantify8(input_tensor):
    
    temp4 = []
    for i in range(len(input_tensor)):
        temp3 = []
        for j in range(len(input_tensor[i])):
            temp2 = []
            for k in range(len(input_tensor[i][j])):
                temp1 = []
                for z in range(len(input_tensor[i][j][k])):
                    a = Quantify8_num(input_tensor[i][j][k][z])
                    temp1.append(a)
                temp2.append(temp1)
            temp3.append(temp2)
        temp4.append(temp3)
        
        temp4 = np.array(temp4)
        return temp4

In [50]:
def Compare(in1,in2,threshold):
    
    temp4 = []
    for i in range(len(in2)):
        temp3 = []
        for j in range(len(in2[i])):
            temp2 = []
            for k in range(len(in2[i][j])):
                temp1 = []
                for z in range(len(in2[i][j][k])):
                    if in2[i][j][k][z] < threshold:
                        temp1.append(0)
                    else:
                        temp1.append(in1[i][j][k][z])
                temp2.append(temp1)
            temp3.append(temp2)
        temp4.append(temp3)
        
        return temp4

In [55]:
input_img    = Input((227,227,3))

# x1           = Lambda(Quantify16)(input_img)
x1           = Quantify16(input_img)
x2           = Quantify10(input_img)
conv1_16     = Conv2D(96,(11,11),strides=4,activation=None)(x1)
conv1_10     = Conv2D(96,(11,11),strides=4,activation=None)(x2)
conv1        = Compare(conv1_16,conv1_10,threshold=-4.04)
norm1        = BatchNormalization()(conv1)
maxpool1     = MaxPooling2D(pool_size=(3,3),strides=(2,2))(norm1)

x1           = Quantify16(maxpool1)
x2           = Quantify9(maxpool1)
x1           = ZeroPadding2D((2,2))(x1)
x2           = ZeroPadding2D((2,2))(x2)
conv2_16     = Conv2D(256,(5,5),padding='same',activation=None)(x1)
conv2_9      = Conv2D(256,(5,5),padding='same',activation=None)(x2)
conv2        = Compare(conv2_16,conv2_9,threshold=-28.9)
norm2        = BatchNormalization()(conv2)
maxpool2     = MaxPooling2D(pool_size=(3,3),strides=(2,2))(norm2)

x1           = Quantify16(maxpool2)
x2           = Quantify9(maxpool2)
x1           = ZeroPadding2D((1,1))(x1)
x2           = ZeroPadding2D((1,1))(x2)
conv3_16     = Conv2D(384,(3,3),padding='same',activation=None)(x1)
conv3_9      = Conv2D(384,(3,3),padding='same',activation=None)(x2)
conv3        = Compare(conv3_16,conv3_9,threshold=-5.46)

x1           = Quantify16(conv3)
x2           = Quantify9(conv3)
x1           = ZeroPadding2D((1,1))(x1)
x2           = ZeroPadding2D((1,1))(x2)
conv4_16     = Conv2D(384,(3,3),padding='same',activation=None)(x1)
conv4_9      = Conv2D(384,(3,3),padding='same',activation=None)(x2)
conv4        = Compare(conv4_16,conv4_9,threshold=-0.36)

x1           = Quantify16(conv4)
x2           = Quantify9(conv4)
x1           = ZeroPadding2D((1,1))(x1)
x2           = ZeroPadding2D((1,1))(x2)
conv5_16     = Conv2D(256,(3,3),padding='same',activation=None)(x1)
conv5_9      = Conv2D(256,(3,3),padding='same',activation=None)(x2)
conv5        = Compare(conv5_16,conv5_9,threshold=0)
maxpool5     = MaxPooling2D(pool_size=(3,3),strides=(2,2))(conv5)

out          = Flatten()(maxpool5)
fullyconnect = Dense(4096,activation='relu')(out)
dropout      = Dropout(0.5)(fullyconnect)

fullyconnect = Dense(4096,activation='relu')(dropout)
dropout      = Dropout(0.5)(fullyconnect)

fullyconnect = Dense(10,activation='softmax')(dropout)
out          = Dropout(0.5)(fullyconnect)

InvalidArgumentError: You must feed a value for placeholder tensor 'input_3' with dtype float and shape [?,227,227,3]
	 [[node input_3 (defined at /home/wlchin/yufen/cnn/lib/python3.5/site-packages/keras/backend/tensorflow_backend.py:517)  = Placeholder[dtype=DT_FLOAT, shape=[?,227,227,3], _device="/job:localhost/replica:0/task:0/device:CPU:0"]()]]

Caused by op 'input_3', defined at:
  File "/usr/lib/python3.5/runpy.py", line 184, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/wlchin/yufen/cnn/lib/python3.5/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/wlchin/yufen/cnn/lib/python3.5/site-packages/traitlets/config/application.py", line 664, in launch_instance
    app.start()
  File "/home/wlchin/yufen/cnn/lib/python3.5/site-packages/ipykernel/kernelapp.py", line 583, in start
    self.io_loop.start()
  File "/home/wlchin/yufen/

In [ ]:
model = Model(inputs = input_img, outputs = out)

In [ ]:
sgd = SGD(lr=0.01, momentum=0.0, nesterov=False)

In [ ]:
model.compile(optimizer = 'sgd', loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [ ]:
model.summary()

In [ ]:
earlystop = EarlyStopping(monitor = 'num3_acc', patience = 5, verbose = 1, mode = 'auto' )
tensorboard = TensorBoard(log_dir = "./logs", histogram_freq = 0, write_graph = True)
callback_list = [tensorboard, earlystop]

In [ ]:
print('Train--------')

model.fit(x_train, y_train, batch_size = 100, epochs = 50, callbacks = callback_list)　#訓練模型